<a href="https://colab.research.google.com/github/leniwiecc/PassGenerator/blob/main/Klocek_SQLAlchemy_cz3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Cześć, jak tam? Magiczne kociołki zainstalowane? Jakieś pierwsze tabelki już zrobione? Dziś stworzymy praktyczny przykład:  sklep internetowy. Stworzymy bazę danych (dość uproszczoną). Zrobimy tabelę: użytkowników, zamówień i produktów a do tego relacje 1 do wielu i wiele do wielu: jeden użytkownik może mieć  wiele zamówień ale każde zamówienie ma tylko 1 właściciela (1 do wielu). Każde zamówienie może zawierać wiele produktów i każdy produkt może być w wielu zamówieniach (relacja wiele do wielu). To co, zaczynamy. Zaczniemy od wykreownania odpowienich tabel:

In [ ]:
produkts_orders= db.Table('produkts_orders',
    db.Column('order_id', db.Integer,   db.ForeignKey('ORDERS.id')),
    db.Column('produkt_id', db.Integer, db.ForeignKey('PRODUKTS.id'))                        
    )

class USERS(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    username= db.Column(db.String(255), nullable=False, unique=True)
    password= db.Column(db.String(255), nullable=False, unique=True)
    mail=db.Column(db.String(255), nullable=False, unique=True)
    orders =  db.relationship('ORDERS', backref='owner_id', lazy='select')
    
class PRODUKTS(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    name= db.Column(db.String(255), nullable=False, unique=True)
    price= db.Column(db.Float(), nullable=False)
     
    
class ORDERS(db.Model):
    id=db.Column(db.Integer, primary_key=True)
    owner= db.Column(db.Integer,  db.ForeignKey('USERS.id'), nullable = False)
    produkts = db.relationship('PRODUKTS', secondary = 'produkts_orders', backref=db.backref('produkts'), lazy='select')

db.create_all()

Tworzenie tabel omawialiśmy wcześniej więc nie będę się powtarzał. Przyjrzyjmy się tabelom USERS i ORDERS. W tabeli ORDERS mamy kolumnę owner (właściciel, użytkownik, który stworzył to zamówienie): deklarujemy że będzie to kolumna int, db.ForeignKey('USERS.id') -> powoduje że z tabeli USERS będziemy brać pole id (które tu będziemy wstawić). 
FOREIGN KEY znany jako klucz obcy to pewnego rodzaju odnośnik łączący tabelę w którym występuje klucz obcy z inną tabelą. Klucz obcy zapobiega wszelkim operacjom, które mogłyby zerwać taką więź międzu tabelami. Również klucz obcy nie może być wartością dowolną. Musi wskazyswać na konkretny wiersz w drugiej tabeli. Klucz obcy można ustalić w trakcie tworzenia nowej tabeli lub dla już istniejącej tabeli.Natomiast w tabeli USERS mamy taką nowość: deklarujemy relacje: orders =  db.relationship('ORDERS', backref='owner_id', lazy='select'). ORDERS to tabela z którą się łączymy, parametr backref - to nazwa po jakiej się będziemy odwoływać z zapytaniach (o tym później), parametr lazy określa sposób ładowania danych przy łączeniu tabel. Jeżeli chcecie zagłębić temat to odsyłam do dokumentacji. 
https://docs.sqlalchemy.org/en/14/orm/loading_relationships.htmlDodajmy paru użytkowników i kilka zamówień żeby sprawdzić jak to działa:

In [ ]:
user1 = USERS(username='Tomek', password='pass', mail='tomek@hm.com')
user2 = USERS(username='Kasia', password='kasia1234', mail='kasia@buziaczek.pl')
db.session.add([user1, user2))
db.session.commit()

Zamówienie możemy dodać na dwa sposoby:

In [ ]:
user = USERS.query.get(2) ## pobieramy danego użytkownika, możemy to też zrobić metodą filer_by
zam1 = ORDERS(owner=user.username)

## a inny spobób to po prostu wpisanie konkretnego numeru id użytkownika:
zam2 = ORDERS(owner=1)
zam3 = ORDERS(owner=2)

db.session.add([zam1, zam2, zam3])
db.session.commit()


To teraz zobaczmy co nam się udało wstawić do bazy:

In [ ]:
users=USERS.query.all()
for user in users:
   print('Użytkownik: {}, a jego zamówienia to:'.format(user.username))
   orders = ORDERS.query.filter(ORDERS.owner == user.id).all()
   for order in orders:
        print('\t- Zamówienie o nr: {}, id_użytkownika: {}, imię użytkownika: {}'.format(order.id, order.owner, order.owner_id.username))

Powyższy kod powinien zwrócić coś w stylu:

```
Użytkownik: rafal, a jego zamówienia to:
	- Zamówienie o nr: 1, id_użytkownika: 1, imię użytkownika: rafal
	- Zamówienie o nr: 2, id_użytkownika: 1, imię użytkownika: rafal
Użytkownik: tomek, a jego zamówienia to:
	- Zamówienie o nr: 3, id_użytkownika: 2, imię użytkownika: tomek
```
zmienna owner.order zwraca nam id użytkownika natomiast owner_id (o którym wczesniej wspominałem) odnosi się do tabeli USERS i po kropce możemy odnosić się do każdej wartości USERS'a (czyli u nas może to być: username, password, mail).
Teraz dodajmy kilka produktów do tabeli PRODUKTS:


In [ ]:
o = PRODUKTS(name='ogórek', price=3.99)
z = PRODUKTS(name='ziemniak', price=0.99)
c = PRODUKTS(name='CD', price=12.55)

db.session.add([o, z, c])
db.session.commit()

Teraz pora na omówienie relacji wiele do wielu. Jak już się domyślacie nie możemy tego zrobić tak samo jak powyżej. Musimy zrobić dodatkową tabelę w której będziemy trzmyać id_zamówienia oraz id_produktu. Wybierając z tej tableli wszystkie wpisy o danym id_zamówienia otrzymamy listę produktów z tego zamówienia;a  filtrując dane przez dane id_produktu zobaczymy listę zamówień, które zawierają dany produkt. Taką tabelę deklarujemy trochę inaczej: 

In [ ]:
produkts_orders= db.Table('produkts_orders', ## nazwa tabeli
    db.Column('order_id', db.Integer,   db.ForeignKey('ORDERS.id')), ## nazwa kolumny, typ danych i odwołanie do 
    db.Column('produkt_id', db.Integer, db.ForeignKey('PRODUKTS.id'))                        
    )

# musimy jeszcze dodać relację w Klasie ORDERS:
produkts = db.relationship('PRODUKTS', secondary = 'produkts_orders', backref=db.backref('produkts'), lazy='select')



w nawiasie w pierwszym miejscu mamy odniesienie do drugiej tabeli, parametr secondary określa tabelę do której jest to odwołanie.
Dodajmy jakieś produkty do zamówienia:

In [ ]:
 
c = PRODUKTS.query.get(1)   #pobieramy konkretny produkt
zam = ORDERS.query.get(1)   #pobieramy konkrente zamówienie

zam.produkts.append(c)  #zam.produkts jest listą produktów danego zamówienia więc dołączamy do niego produkt c

db.session.commit()

To teraz zobaczmy co mamy w koszyku:

In [ ]:
zam = ORDERS.query.get(1)
print('Lista produktów z zamówienia o id: {}'.format(zam.id))
for produkt in zam.produkts:
     print(' - {}, {}'.format(produkt.name, produkt.price))


Powyższy kod powinien zwrócić coś takiego:

```
Lista produktów z zamówienia o id: 1
 - ziemniak, 0.99
 - ogórek, 3.99
```



I to by było wszystko co chciałem Wam przekazać. Bierzcie i korzystajcie z tego bez ograniczeń :) Jak coś to pisać śmiało, RafalK.